In [3]:
# Import necessary libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.metrics import MeanIoU
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

# Constants
IMG_SIZE = 128
BATCH_SIZE = 16
EPOCHS = 15

# Preprocess images and generate masks
def preprocess_image(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0

    # Generate a binary mask based on a simple threshold on the red channel
    mask = tf.where(image[..., 0] > 0.5, 1.0, 0.0)
    mask = tf.expand_dims(mask, axis=-1)  # Add channel dimension
    return image, mask

# Load Oxford Flowers Dataset
def load_data():
    dataset, info = tfds.load("oxford_flowers102", as_supervised=True, with_info=True)
    train_data = dataset["train"].map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    test_data = dataset["test"].map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    return train_data, test_data

# Load dataset
train_data, test_data = load_data()
train_data = train_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data = test_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Define U-Net Model
def unet_model(input_size=(IMG_SIZE, IMG_SIZE, 3)):
    inputs = layers.Input(input_size)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(p1)
    c2 = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation="relu", padding="same")(p2)
    c3 = layers.Conv2D(256, (3, 3), activation="relu", padding="same")(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(512, (3, 3), activation="relu", padding="same")(p3)
    c4 = layers.Conv2D(512, (3, 3), activation="relu", padding="same")(c4)

    # Decoder
    u5 = layers.Conv2DTranspose(256, (3, 3), strides=(2, 2), padding="same")(c4)
    u5 = layers.concatenate([u5, c3])
    c5 = layers.Conv2D(256, (3, 3), activation="relu", padding="same")(u5)
    c5 = layers.Conv2D(256, (3, 3), activation="relu", padding="same")(c5)

    u6 = layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding="same")(c5)
    u6 = layers.concatenate([u6, c2])
    c6 = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(u6)
    c6 = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(c6)

    u7 = layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding="same")(c6)
    u7 = layers.concatenate([u7, c1])
    c7 = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(u7)
    c7 = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(c7)

    outputs = layers.Conv2D(1, (1, 1), activation="sigmoid")(c7)

    model = models.Model(inputs, outputs)
    return model

model = unet_model()
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy", MeanIoU(num_classes=2)]
)

# Train the model
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=EPOCHS
)

# Evaluate the model
val_loss, val_acc, val_mean_iou = model.evaluate(test_data)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}, Mean IoU: {val_mean_iou}")

# Prediction and visualization
def display_results(image, true_mask, pred_mask):
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 3, 1)
    plt.title("Image")
    plt.imshow(image)

    plt.subplot(1, 3, 2)
    plt.title("True Mask")
    plt.imshow(true_mask[..., 0], cmap="gray")

    plt.subplot(1, 3, 3)
    plt.title("Predicted Mask")
    plt.imshow(pred_mask[..., 0], cmap="gray")

    plt.show()

# Predict and visualize
sample_image, sample_mask = next(iter(test_data))
pred_mask = model.predict(sample_image)

for i in range(10):
    display_results(sample_image[i].numpy(), sample_mask[i].numpy(), pred_mask[i])


2025-01-25 16:33:40.641227: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


/usr/lib/python3/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:  67%|██████▋   | 2/3 [4:20:33<00:05,  5.70s/ url]
Extraction completed...: 0 file [4:21:18, ? file/s]
Dl Completed...:  67%|██████▋   | 2/3 [4:21:18<2:10:39, 7839.36s/ url]


ChunkedEncodingError: ('Connection broken: IncompleteRead(187924216 bytes read, 156938293 more expected)', IncompleteRead(187924216 bytes read, 156938293 more expected))